In [1]:
import numpy as np
import joblib
from joblib import Parallel, delayed

In [3]:
joblib.cpu_count()

12

In [38]:
# parallelize a for loop that counts how many even numbers exist in an array

# given an array, count how many even numbers are present in the array (n) and return a list containing n elements

In [73]:
a = np.random.randint(10, size=10)
a

array([1, 7, 5, 2, 8, 0, 4, 6, 0, 1])

In [2]:
def even_numbers(array):
    count = 0
    for number in array:
        if number % 2 == 0:
            count += 1
    return count

In [75]:
%%time

result = [k for k in [even_numbers(a)] for i in range(even_numbers(a))]

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 19.1 µs


In [76]:
result

[6, 6, 6, 6, 6, 6]

In [78]:
%%time

Parallel(n_jobs=-1)(delayed(even_numbers)(a) for i in range(even_numbers(a)))

CPU times: user 73.6 ms, sys: 0 ns, total: 73.6 ms
Wall time: 196 ms


[6, 6, 6, 6, 6, 6]

[Shared Memory Semantics](https://joblib.readthedocs.io/en/latest/parallel.html#shared-memory-semantics)

In [79]:
%%time

Parallel(n_jobs=-1, require='sharedmem')(delayed(even_numbers)(a) for i in range(even_numbers(a)))

CPU times: user 6.34 ms, sys: 0 ns, total: 6.34 ms
Wall time: 4.28 ms


[6, 6, 6, 6, 6, 6]

In [21]:
from multiprocessing import Pool, Process, Queue

In [33]:
%%time
with Pool() as pool:
    results = pool.map(even_numbers, [np.random.randint(10, size=10) for _ in range(100)])

CPU times: user 18 ms, sys: 52.9 ms, total: 70.9 ms
Wall time: 73.6 ms


In [34]:
%%time
results = list(map(even_numbers, [np.random.randint(10, size=10) for _ in range(100)]))

CPU times: user 8.06 ms, sys: 4.06 ms, total: 12.1 ms
Wall time: 9.81 ms


In [24]:
def even_numbers_(in_queue, out_queue):
    my_list = in_queue.get()
    count = 0
    for item in my_list:
        if item % 2 == 0:
            count += 1
    
    out_queue.put(count)


In [25]:
in_queue = Queue()
for _ in range(100):  
    in_queue.put(np.random.randint(10, size=10))

out_queue = Queue() 


In [26]:
processes = []
for _ in range(in_queue.qsize()):
    processes.append(
        Process(target=even_numbers_, args=(in_queue, out_queue))
    )

In [27]:
%%time
for process in processes:
    process.start()

for process in processes:
    process.join()

for process in processes:
    process.close()

CPU times: user 70.1 ms, sys: 145 ms, total: 215 ms
Wall time: 208 ms


In [28]:
results = [out_queue.get() for _ in range(out_queue.qsize())]

CPU times: user 3.4 ms, sys: 37 µs, total: 3.44 ms
Wall time: 1.79 ms


In [36]:
len(results)

100